# Neighborhoods in Toronto

This notebook is a Coursera Capstone's assignment to segment and cluster neighborhoods in Toronto, Canada.

## Section 1: Obtaining and Cleaning Data
The section below is the coding for obtaining data from Wikipedia, and performing relevant data cleaning tasks to achieve the desired DataFrame.

__Note1:__ The default data on Wikipedia has been formatted differently than the instruction seems to be taking it for. There is no duplicate of Postal Codes in the table anymore, making it irrelevant to perform the task that look for duplicated Postal Codes and append neighborhoods separating by a comma.

__Note2:__ The default data on Wikipedia also no longer contain any instances where a cell has a borough but a Not assigned neighborhood. Thus, the task to look for this instance and assigned neighborhood to be the same as borough will also not be carried out.

In [148]:
#importing relevant modules

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [211]:
#Use read_html to get data from Wikipedia
toronto_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_neigh = pd.read_html(toronto_url, match='Postal Code')

#read_html returns list of dataframe, choosing 1st on the list to get the table
toronto_neigh = toronto_neigh[0]

#rename column headers
toronto_neigh.columns = ['PostalCode','Borough','Neighborhood']
print("Shape of Table: {} rows and {} columns.".format(toronto_neigh.shape[0]
                                                       ,toronto_neigh.shape[1]))
toronto_neigh.head()

Shape of Table: 180 rows and 3 columns.


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [212]:
#Drop rows that contain 'Not assigned' in Borough column
toronto_neigh.drop(toronto_neigh[toronto_neigh.Borough == 'Not assigned']
                   .index, inplace=True)

#sort values by PostalCode and reset index
toronto_neigh.sort_values(by = ['PostalCode'], ascending=False)
toronto_neigh.reset_index(drop = True, inplace =True)
      
print("Shape of Table after dropping 'Not assigned' Borough: \
{} rows and {} columns.".format(toronto_neigh.shape[0]
                                        ,toronto_neigh.shape[1]))
toronto_neigh.head(12)

Shape of Table after dropping 'Not assigned' Borough: 103 rows and 3 columns.


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
#A check to confirm that there is no rows with Not assigned in either
#Borough or Neighborhood
print("Number of rows that has 'Not assigned' as Borough: "
      + str(len(toronto_neigh[toronto_neigh.Neighborhood == 'Not assigned'])))
print("Number of rows that has 'Not assigned' as Neighborhood: "
      + str(len(toronto_neigh[toronto_neigh.Borough == 'Not assigned'])))

Number of rows that has 'Not assigned' as Borough: 0
Number of rows that has 'Not assigned' as Neighborhood: 0


In [5]:
#Print out the shape of the final table after cleaning the data
print("Shape of the final table: {} rows and {} columns.".format(
    toronto_neigh.shape[0],toronto_neigh.shape[1]))

Shape of the final table: 103 rows and 3 columns.


## Section2: Obtaining Coordinates from Postal Code
This section highlights the process of obtaining coordinates of each Postal Code in Toronto. Due to issue with geocoder package, which fail to get coordinates from the Postal Code, a provided CSV file was used to get the coordinates instead.

The process below involves reading CSV file into a DataFrame, and merging the new DataFrame into the existing toronto_neigh which contain cleaned neighborhood data. I then dropped the 'Postal Code' column that was appended to the main DataFrame by the merging process, which resulted in a table similar to the example.

In [6]:
#Read the give coordinates CSV into a DataFrame
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Merge coords DataFrame to existing DataFrame
toronto_neigh = toronto_neigh.merge(coords, left_on='PostalCode',
                                    right_on='Postal Code')

#Dropping 'Postal Code' column from coords DataFrame
toronto_neigh.drop(['Postal Code'], axis=1, inplace = True)
toronto_neigh

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Section 3: Clustering and Visualisation
This section looks to cluster Toronto's neighborhoods based on Point of Interests venues obtained from Foursquare API. Each cluster is labeled on the map of Toronto to visualise similar neighborhood.

The first part of this section is to create get URL to access and obtain venues data from Foursquare. The data is then cleaned and transformed to get the top 10 most common venues for each neighborhood.

The second part is then looking to using k-Means clustering to find similar neighborhood based on similarity of venues within the set radius.

Lastly, the third part maps the neighborhood and relevant cluster label on the map of Toronto to visualise similarity of each neighborhood.

In [84]:
#importing relevant packages
import folium
import requests
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Section 3 - Part 1: Get Venues from Foursquare and Clean/Transform Data
Part 1 below gets venues data from Foursquare through its API and perform data cleaning/transforming to shape the data into a workable format.

In [163]:
#creating url for Foursquare API to get venues
CLIENT_ID = 'H4XUBXBTKFVT5Q1ZTIBLIBMPPTXTUOH0LHK4T0TRKPRHKOQW' # your Foursquare ID
CLIENT_SECRET = 'LEDR45BDQPMK0JLIQBUVQQDFYA5U350WO0MFDZG2AN2IZSWK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#limiting number of venues to 100 and radius of 500
radius = 500
LIMIT = 100

In [27]:
#storing venues json to results
results = requests.get(url).json()

In [164]:
#create a function to generate get url to obtain venues data from Foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [183]:
#Run the getNearbyVenues function to get venues data and store in dataframe
toronto_venues = getNearbyVenues(names=toronto_neigh['Neighborhood'],
                                   latitudes=toronto_neigh['Latitude'],
                                   longitudes=toronto_neigh['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [184]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], 
                                prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
n_col = toronto_onehot.columns.get_loc('Neighborhood')
if n_col == 0:
    fixed_columns = list(toronto_onehot.columns[:])
else:
    fixed_columns = [toronto_onehot.columns[n_col]] + list(toronto_onehot.columns[0:n_col]) + list(toronto_onehot.columns[n_col+1:])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2136, 269)


,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [136]:
#Group data into each neighborhood by mean value
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0

In [137]:
#a function to get the top most common venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [185]:
#set number of top venues to be used
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Coffee Shop,Pub,Sandwich Place,Gym,Airport Terminal,Colombian Restaurant,Farm
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Supermarket,Middle Eastern Restaurant,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Thai Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store


### Section 3 - Part 2: k-Means Clustering
Part 2 below is utilising k-Means clustering method to find similar neighborhood based on common venues within the set radius. The cluster labels obtain from clustering procedure are then appended back onto the DataFrame as a step to prepare data before mapping them.

In [186]:
#set number of k-means cluster
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

#Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 3, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1,
       3, 1, 1, 1, 1, 1, 1, 3])

In [188]:
#append clustering labels to data
toronto_merged = toronto_neigh

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Bus Stop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Event Space,Miscellaneous Shop,Falafel Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium


### Section 3 - Part 3: Mapping Neighborhoods
The last part here is set to map neighborhoods on map of Toronto using colors to identify similar neighborhood. Tables below also display neighborhood for each cluster labels.

__Note:__ There are three "NaN" neighborhoods that do not have any nearby venues on Foursquare. This could be a result of the set radius being too small to include venues, or the coordinates of these neighborhood center around area where there isn't much. These "NaN" neighborhoods are plotted as black circle on the map.

In [190]:
# set color scheme for the clusters
x = np.arange(kclusters)
print(x)
print(range(kclusters))
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
map_toronto = folium.Map(location=[43.7087, -79.35544], zoom_start=10)
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #if loop was used to check for NaN clusters and set color to black
    if np.isnan(cluster):
        cluster = 'No Data'
        colour = 'black'
    else:
        colour = rainbow[int(cluster)-1]
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colour,
        fill=True,
        fill_color=colour,
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

[0 1 2 3 4]
range(0, 5)
[array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]


In [193]:
#Neighborhood with cluster label 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,
                     toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,Scarborough Village,0.0,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
85,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",0.0,Park,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
91,Downtown Toronto,Rosedale,0.0,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [194]:
#Neighborhood with cluster label 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,
                     toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,1.0,Food & Drink Shop,Park,Bus Stop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,North York,Victoria Village,1.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,"Regent Park, Harbourfront",1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,North York,"Lawrence Manor, Lawrence Heights",1.0,Clothing Store,Accessories Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Event Space,Miscellaneous Shop,Falafel Restaurant
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",1.0,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
6,Scarborough,"Malvern, Rouge",1.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
7,North York,Don Mills,1.0,Gym,Restaurant,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Bike Shop
8,East York,"Parkview Hill, Woodbine Gardens",1.0,Pizza Place,Pharmacy,Gastropub,Pet Store,Gym / Fitness Center,Breakfast Spot,Bank,Intersection,Athletics & Sports,Fast Food Restaurant
9,Downtown Toronto,"Garden District, Ryerson",1.0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Japanese Restaurant,Hotel,Pizza Place,Middle Eastern Restaurant,Lingerie Store
10,North York,Glencairn,1.0,Bakery,Pub,Japanese Restaurant,Pizza Place,Italian Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [195]:
#Neighborhood with cluster label 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,
                     toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,"Humberlea, Emery",2.0,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Field
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2.0,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Field


In [196]:
#Neighborhood with cluster label 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,
                     toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,Caledonia-Fairbanks,3.0,Park,Women's Store,Bar,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
35,East York,"East Toronto, Broadview North (Old East York)",3.0,Intersection,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
61,Central Toronto,Lawrence Park,3.0,Park,Bus Line,Swim School,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
64,York,Weston,3.0,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
66,North York,York Mills West,3.0,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store


In [197]:
#Neighborhood with cluster label 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,
                     toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",4.0,Brewery,Yoga Studio,Dim Sum Restaurant,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore


In [204]:
#Neighborhood with NaN as cluster label, this is due to no point of interests
#are located within the set radius of the neighborhood.
toronto_merged[toronto_merged.isna().any(axis=1)].loc[:,toronto_merged.columns[[1]+[2] + list(range(5,
                        toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,"Islington Avenue, Humber Valley Village",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,North York,"York Mills, Silver Hills",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Scarborough,Upper Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


__----------------------------------------------------------------------------------------------------------------------------------__